# Neural Network Model

 * [Imports and Model Specific Cleaning](#IMSC)
 * [Model](#mod)
 * [Save Model](#sm)

<a id='IMSC'></a>
## Imports and Model Specific Cleaning

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV

from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

In [2]:
# set view columns to 250
pd.set_option('display.max_columns',250)

In [3]:
# read in data
accidents = pd.read_csv('../project-5/data/severity_model_data/selected.csv',index_col = 0)

In [4]:
# create balanced classes (severity 1 is the smallest with 29312 instances)
sev_1 = accidents[accidents['severity'] == 1]
sev_2 = accidents[accidents['severity'] == 2].head(29312)
sev_3 = accidents[accidents['severity'] == 3].head(29312)
sev_4 = accidents[accidents['severity'] == 4].head(29312)
selected = pd.concat([sev_1,sev_2,sev_3,sev_4], axis = 0).reset_index()

In [5]:
# print only columns with nulls and their null values
for col in selected.columns:
    if selected[col].isnull().sum() > 0:
        print(col)
        print(selected[col].isnull().sum())

temperaturef
1810
humidity
2086
visibilitymi
2084
wind_speedmph
16969
precipitationin
76191
weather_condition
2172
wind_chillf
73105


In [6]:
# map side
selected['side'] = selected['side'].map({'R':1,'L':0})

In [7]:
# fill NaN values for side column
selected['side'] = selected['side'].fillna(1)

In [8]:
# fill NaN values for precipitationin column
selected['precipitationin'] = selected['precipitationin'].fillna(0)

In [9]:
# fill NaN values for temperaturef column
selected['temperaturef'].fillna(selected['temperaturef'].median(), inplace=True)

In [10]:
# fill NaN values for humidity column
selected['humidity'].fillna(100.0, inplace=True)

In [11]:
# fill NaN values for visibilitymi column
selected['visibilitymi'].fillna(selected['visibilitymi'].median(), inplace=True)

In [12]:
# fill NaN values for weather_condition column
selected['weather_condition'].fillna('Fair', inplace=True)

In [13]:
# caluculate wind_chillf column for nan values if temp and wind chill are provided
selected['wind_chillf'] = selected.apply(lambda row: row[
    'temperaturef'] - row['wind_speedmph'] if np.isnan(row['wind_chillf']) else row['wind_chillf'], axis=1)

In [14]:
# fill NaN values for wind_speedmph column
selected['wind_speedmph'].fillna(0, inplace=True)

In [15]:
# fill NaN values for wind_chillf column
selected['wind_chillf'].fillna(selected['temperaturef'], inplace=True)

In [16]:
# dummify columns
selected = pd.get_dummies(selected,columns = ['start_hour','month','end_hour','county','weather_condition'])

In [17]:
# drop index columns
selected.drop(columns = ['index','county.1'],inplace = True)

In [18]:
selected.head()

,severity,temperaturef,humidity,visibilitymi,wind_speedmph,precipitationin,traffic_signal,side,crossing,junction,stop,distancemi,wind_chillf,traffic_signal.1,sunrise_sunset,civil_twilight,nautical_twilight,astronomical_twilight,total_duration,start_hour_0,start_hour_1,start_hour_2,start_hour_3,start_hour_4,start_hour_5,start_hour_6,start_hour_7,start_hour_8,start_hour_9,start_hour_10,start_hour_11,start_hour_12,start_hour_13,start_hour_14,start_hour_15,start_hour_16,start_hour_17,start_hour_18,start_hour_19,start_hour_20,start_hour_21,start_hour_22,start_hour_23,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,end_hour_0,end_hour_1,end_hour_2,end_hour_3,end_hour_4,end_hour_5,end_hour_6,end_hour_7,end_hour_8,end_hour_9,end_hour_10,end_hour_11,end_hour_12,end_hour_13,end_hour_14,end_hour_15,end_hour_16,end_hour_17,end_hour_18,end_hour_19,end_hour_20,end_hour_21,end_hour_22,end_hour_23,county_AcadiaLA,county_AccomackVA,county_AdaID,county_AdairIA,county_AdamsCO,county_AdamsIA,county_AdamsID,county_AdamsIN,county_AdamsNE,county_AdamsOH,county_AdamsPA,county_AdamsWA,county_AdamsWI,county_AddisonVT,county_AikenSC,county_AlachuaFL,county_AlamanceNC,county_AlamedaCA,county_AlamosaCO,county_AlbanyNY,county_AlbanyWY,county_AlbemarleVA,county_AlconaMI,county_AlexanderNC,county_AlexandriaVA,county_AllamakeeIA,county_AlleganMI,county_AlleganyMD,county_AlleganyNY,county_AlleghanyVA,county_AlleghenyPA,county_AllenIN,county_AllenOH,county_AlpenaMI,county_AlpineCA,county_AmadorCA,county_AmeliaVA,county_AmherstVA,county_AndersonKY,county_AndersonSC,county_AndersonTN,county_AndrewMO,county_AndroscogginME,county_Anne ArundelMD,county_AnokaMN,county_AnsonNC,...,county_WillIL,county_WilliamsOH,county_WilliamsonIL,county_WilliamsonTN,county_WilliamsonTX,county_WilsonNC,county_WilsonTN,county_WindhamCT,county_WindhamVT,county_WindsorVT,county_WinnebagoIA,county_WinnebagoIL,county_WinnebagoWI,county_WinneshiekIA,county_WinonaMN,county_WiseVA,county_WolfeKY,county_WoodOH,county_WoodWI,county_WoodWV,county_WoodburyIA,county_WoodfordKY,county_WorcesterMA,county_WorcesterMD,county_WorthGA,county_WorthIA,county_WrightIA,county_WrightMN,county_WrightMO,county_WyandotOH,county_WyandotteKS,county_WyomingNY,county_WyomingPA,county_WytheVA,county_YadkinNC,county_YakimaWA,county_YalobushaMS,county_YamhillOR,county_YatesNY,county_YavapaiAZ,county_YellowstoneMT,county_YoakumTX,county_YoloCA,county_YorkME,county_YorkNE,county_YorkPA,county_YorkSC,county_YorkVA,county_YubaCA,county_YumaAZ,county_YumaCO,weather_condition_Blowing Dust,weather_condition_Blowing Dust / Windy,weather_condition_Blowing Sand,weather_condition_Blowing Snow,weather_condition_Clear,weather_condition_Cloudy,weather_condition_Cloudy / Windy,weather_condition_Drizzle,weather_condition_Fair,weather_condition_Fair / Windy,weather_condition_Fog,weather_condition_Fog / Windy,weather_condition_Freezing Rain / Windy,weather_condition_Hail,weather_condition_Haze,weather_condition_Haze / Windy,weather_condition_Heavy Drizzle,weather_condition_Heavy Rain,weather_condition_Heavy Rain / Windy,weather_condition_Heavy Snow,weather_condition_Heavy Snow / Windy,weather_condition_Heavy T-Storm,weather_condition_Heavy T-Storm / Windy,weather_condition_Heavy Thunderstorms and Rain,weather_condition_Heavy Thunderstorms and Snow,weather_condition_Light Drizzle,weather_condition_Light Drizzle / Windy,weather_condition_Light Freezing Drizzle,weather_condition_Light Freezing Fog,weather_condition_Light Freezing Rain,weather_condition_Light Haze,weather_condition_Light Ice Pellets,weather_condition_Light Rain,weather_condition_Light Rain / Windy,weather_condition_Light Rain Shower,weather_condition_Light Rain Shower / Windy,weather_condition_Light Rain Showers,weather_condition_Light Rain with Thunder,weather_condition_Light Sleet,weather_condition_Light Snow,weather_condition_Light Snow / Windy,weather_condition_Light Snow Showers,weather_condition_Light Thunderstorms 

<a id='mod'></a>
## Model
### Keras

In [17]:
# create a features variable
features = [col for col in selected.columns if col not in ['severity']]

In [18]:
# set X and y variables
X_nn = selected[features]
y_nn = selected[['severity']]

In [19]:
# subtract 1 from severity so it to_categorical will work
y_nn['severity'] = y_nn['severity'] -1

<ipython-input-19-2648100aef3a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_nn['severity'] = y_nn['severity'] -1


In [20]:
# train_test_split data
X_nn_train,X_nn_test,y_nn_train,y_nn_test = train_test_split(X_nn,y_nn,random_state = 42)

In [21]:
# to_categorical y data since it is a multiclassification
y_nn_train = to_categorical(y_nn_train,4)
y_nn_test = to_categorical(y_nn_test,4)

In [22]:
# scale data
sc = StandardScaler()
X_nn_train_sc = sc.fit_transform(X_nn_train)
X_nn_test_sc = sc.transform(X_nn_test)

In [23]:
# define a function that will go in the scikit-learn wrapper w/ arbitrary layers, neurons
def model_func(hidden_layers,layer_odd_neurons = 20,layer_even_neurons = 10, 
               layer_odd_dropout = 0.5,layer_even_dropout=0.5,e_stop = False):
    
    # instantiate Sequential
    model = Sequential()
    
    # set up hidden layers
    for x in range(hidden_layers):
        if x == 0: # input layer
            model.add(Dense(units = layer_even_neurons, activation = 'relu', 
                            input_shape = (X_nn_train_sc.shape[1],)))
        elif x%2 == 0: # even hidden layers
            model.add(Dense(units = layer_even_neurons,activation = 'relu'))
            model.add(Dropout(layer_even_dropout))
        else: # odd hidden layers
            model.add(Dense(units = layer_odd_neurons,activation = 'relu'))
            model.add(Dropout(layer_odd_dropout))
        
    # output layer
    model.add(Dense(4, activation = 'softmax'))
    
    # compile model
    model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
    
    # add EarlyStopping
    if e_stop == True:
        early_nn_stop = EarlyStopping(monitor = 'val_loss',min_delta = 0, 
                                   patience = 5,verbose = 1,mode = 'auto')
    else:
        pass

    return model

# originally used these parameters
#     {'epochs':[100],
#     'hidden_layers': [5,7],
#     'layer_odd_neurons':[8,16],
#     'layer_even_neurons':[16,32],
#     'layer_odd_dropout':[0.3,0.5],
#     'layer_even_dropout':[0.2,0.5],
#     'e_stop': [False,True]}

# got these params as the gs_nn.best_params_ and ran twice to get the best model
params = {
    'e_stop': [True],
    'epochs': [100],
    'hidden_layers': [2],
    'layer_even_dropout': [0.5],
    'layer_even_neurons': [32],
    'layer_odd_dropout': [0.3],
    'layer_odd_neurons': [16]
}

In [24]:
#wrap in a KerasClassifier object
nn = KerasClassifier(build_fn = model_func,epochs = 10,batch_size = 512,verbose = 1)

In [25]:
# set up GridSearch
gs_nn = GridSearchCV(estimator = nn,param_grid = params,cv = 5,n_jobs = -1)

In [26]:
# fit model
gs_nn.fit(X_nn_train_sc, y_nn_train)

Epoch 1/100
172/172 [==============================] - 1s 3ms/step - loss: 1.4402 - accuracy: 0.4787
Epoch 2/100
172/172 [==============================] - 0s 3ms/step - loss: 0.5579 - accuracy: 0.7859
Epoch 3/100
172/172 [==============================] - 0s 3ms/step - loss: 0.4569 - accuracy: 0.8171
Epoch 4/100
172/172 [==============================] - 0s 3ms/step - loss: 0.4158 - accuracy: 0.8283
Epoch 5/100
172/172 [==============================] - 0s 3ms/step - loss: 0.3946 - accuracy: 0.8355
Epoch 6/100
172/172 [==============================] - 1s 3ms/step - loss: 0.3743 - accuracy: 0.8447
Epoch 7/100
172/172 [==============================] - 1s 3ms/step - loss: 0.3561 - accuracy: 0.8529
Epoch 8/100
172/172 [==============================] - 1s 3ms/step - loss: 0.3483 - accuracy: 0.8550
Epoch 9/100
172/172 [==============================] - 1s 3ms/step - loss: 0.3362 - accuracy: 0.8597
Epoch 10/100
172/172 [==============================] - 1s 3ms/step - loss: 0.3252 - accura

GridSearchCV(cv=5,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fdcbd815670>,
             n_jobs=-1,
             param_grid={'e_stop': [True], 'epochs': [100],
                         'hidden_layers': [2], 'layer_even_dropout': [0.5],
                         'layer_even_neurons': [32], 'layer_odd_dropout': [0.3],
                         'layer_odd_neurons': [16]})

In [27]:
# get cross val score
gs_nn.best_score_

0.8740447521209717

In [28]:
# get test score
gs_nn.score(X_nn_test_sc,y_nn_test)

58/58 [==============================] - 0s 2ms/step - loss: 0.4646 - accuracy: 0.8803


0.880253791809082

In [29]:
# get train score
gs_nn.score(X_nn_train_sc,y_nn_train)

172/172 [==============================] - 0s 2ms/step - loss: 0.1903 - accuracy: 0.9046


0.9046351909637451

<a id='sm'></a>
### Save Model

In [ ]:
gs_nn_scores = {
    'train score': gs_nn.score(X_nn_train_sc,y_nn_train),
    'test score': gs_nn.score(X_nn_test_sc,y_nn_test)
}

In [ ]:
nn = pd.DataFrame(gs_nn_scores,index = ['Neural Network'])

In [ ]:
nn.to_csv('./data/severity_model_data/nn_results.csv')

In [ ]:
# save model
gs_nn.best_estimator_.model.save('gs_neural_net')

In [ ]:
# instantiate model
model = Sequential()

In [ ]:
# load model to check to see it works
model = load_model('./gs_neural_net/')